In [1]:
#!/usr/bin/env python3
"""
B站视频数据分析 - 终极精简版
输入BV号，获取三连数据并生成柱状图
"""

import asyncio
import matplotlib.pyplot as plt
from bilibili_api import video

class BilibiliQuickAnalyzer:
    def __init__(self):
        plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
        plt.rcParams['axes.unicode_minus'] = False
    
    async def get_video_data(self, bvid):
        """获取视频数据"""
        try:
            v = video.Video(bvid=bvid)
            info = await v.get_info()
            return info
        except Exception as e:
            print(f"❌ 获取视频失败: {e}")
            return None
    
    def analyze_and_plot(self, bvid):
        """分析并绘图 - 主函数"""
        print(f"🎯 正在分析视频: {bvid}")
        
        # 获取数据
        info = asyncio.run(self.get_video_data(bvid))
        if not info:
            return
        
        # 提取关键数据
        stat = info['stat']
        data = {
            '标题': info['title'][:20] + '...' if len(info['title']) > 20 else info['title'],
            '播放量': stat['view'],
            '点赞数': stat['like'], 
            '投币数': stat['coin'],
            '收藏数': stat['favorite']
        }
        
        # 打印结果
        print(f"\n📊 分析结果:")
        print(f"标题: {info['title']}")
        print(f"UP主: {info['owner']['name']}")
        print(f"播放量: {data['播放量']:,}")
        print(f"点赞数: {data['点赞数']:,}")
        print(f"投币数: {data['投币数']:,}") 
        print(f"收藏数: {data['收藏数']:,}")
        
        # 计算比率
        total = data['点赞数'] + data['投币数'] + data['收藏数']
        print(f"三连总数: {total:,}")
        print(f"点赞率: {data['点赞数']/data['播放量']:.2%}")
        print(f"投币率: {data['投币数']/data['播放量']:.2%}")
        print(f"收藏率: {data['收藏数']/data['播放量']:.2%}")
        
        # 生成柱状图
        self.generate_bar_chart(data, bvid)
    
    def generate_bar_chart(self, data, bvid):
        """生成三连数据柱状图"""
        labels = ['播放量', '点赞数', '投币数', '收藏数']
        values = [data['播放量'], data['点赞数'], data['投币数'], data['收藏数']]
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
        
        plt.figure(figsize=(10, 6))
        bars = plt.bar(labels, values, color=colors, alpha=0.7)
        
        # 设置图表样式
        plt.title(f'B站视频数据分析\n{data["标题"]}\nBV: {bvid}', fontsize=14)
        plt.ylabel('数量')
        
        # 在柱子上显示数值
        for bar, value in zip(bars, values):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(values)*0.01, 
                    f'{value:,}', ha='center', va='bottom', fontweight='bold')
        
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.show()

# 使用示例
if __name__ == "__main__":
    analyzer = BilibiliQuickAnalyzer()
    
    # 直接在这里输入BV号！
    bv_id = "BV1cSnuzYE88"  # 🔥 修改这里的BV号！
    
    analyzer.analyze_and_plot(bv_id)

🎯 正在分析视频: BV1cSnuzYE88


RuntimeError: asyncio.run() cannot be called from a running event loop

In [2]:
# 创建 core.py 内容
core_code = '''
"""
B站视频分析核心模块
"""
import asyncio
import time
from bilibili_api import video

class BilibiliAnalyzer:
    """B站视频分析器"""
    
    def __init__(self):
        self.sequence = []
        self.start_time = time.time()
    
    def _checkpoint(self, step_name):
        """内部断点检查"""
        timestamp = time.time() - self.start_time
        self.sequence.append(1)
        print(f"⏰ [{timestamp:.3f}s] {step_name}: 1")
        return timestamp
    
    async def analyze_video(self, bv_id):
        """分析视频数据"""
        try:
            # 创建视频对象
            self._checkpoint("创建视频对象")
            v = video.Video(bvid=bv_id)
            
            # 获取视频信息
            self._checkpoint("获取API数据")
            info = await v.get_info()
            
            # 提取数据
            self._checkpoint("解析数据")
            title = info['title']
            up_name = info['owner']['name']
            stat = info['stat']
            
            # 整理结果
            result = {
                'bv_id': bv_id,
                'title': title,
                'up_name': up_name,
                'view': stat['view'],
                'like': stat['like'],
                'coin': stat['coin'],
                'favorite': stat['favorite'],
                'share': stat['share'],
                'reply': stat['reply'],
                'sequence': self.sequence.copy(),
                'total_time': time.time() - self.start_time
            }
            
            # 计算比率
            result['like_rate'] = result['like'] / result['view']
            result['coin_rate'] = result['coin'] / result['view']
            result['favorite_rate'] = result['favorite'] / result['view']
            
            self._checkpoint("分析完成")
            return result
            
        except Exception as e:
            print(f"❌ 分析失败: {e}")
            return None

class InteractiveTester:
    """交互式测试器"""
    
    def __init__(self):
        self.analyzer = BilibiliAnalyzer()
    
    async def run_interactive_test(self):
        """运行交互式测试"""
        while True:
            print("\\n" + "="*50)
            bv_id = input("请输入BV号 (输入 'q' 退出): ").strip()
            
            if bv_id.lower() == 'q':
                print("测试结束！")
                break
                
            if not bv_id.startswith('BV'):
                print("❌ 请输入有效的BV号")
                continue
            
            print(f"🎯 正在分析: {bv_id}")
            result = await self.analyzer.analyze_video(bv_id)
            
            if result:
                self._display_result(result)
            else:
                print("❌ 分析失败")
    
    def _display_result(self, result):
        """显示分析结果"""
        print(f"\\n✅ 分析成功!")
        print(f"📺 标题: {result['title']}")
        print(f"👤 UP主: {result['up_name']}")
        print(f"📊 播放量: {result['view']:,}")
        print(f"👍 点赞: {result['like']:,} ({result['like_rate']:.2%})")
        print(f"🪙 投币: {result['coin']:,} ({result['coin_rate']:.2%})")
        print(f"⭐ 收藏: {result['favorite']:,} ({result['favorite_rate']:.2%})")
        print(f"🔢 执行序列: {''.join(str(x) for x in result['sequence'])}")
        print(f"⏱️  总耗时: {result['total_time']:.3f}s")
'''

with open('bilibili_analyzer/src/core.py', 'w', encoding='utf-8') as f:
    f.write(core_code)

print("✅ core.py 创建完成!")

FileNotFoundError: [Errno 2] No such file or directory: 'bilibili_analyzer/src/core.py'

In [3]:
# 第一步：先创建目录结构
import os

# 创建必要的文件夹
folders = [
    'bilibili_analyzer',
    'bilibili_analyzer/src', 
    'data/raw',
    'data/processed',
    'notebooks',
    'docs',
    'tests'
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)
    print(f"📁 创建目录: {folder}")

# 创建空的 __init__.py 文件
init_files = [
    'bilibili_analyzer/__init__.py',
    'bilibili_analyzer/src/__init__.py'
]

for file in init_files:
    with open(file, 'w', encoding='utf-8') as f:
        f.write('')
    print(f"📄 创建文件: {file}")

print("✅ 目录结构创建完成!")

📁 创建目录: bilibili_analyzer
📁 创建目录: bilibili_analyzer/src
📁 创建目录: data/raw
📁 创建目录: data/processed
📁 创建目录: notebooks
📁 创建目录: docs
📁 创建目录: tests
📄 创建文件: bilibili_analyzer/__init__.py
📄 创建文件: bilibili_analyzer/src/__init__.py
✅ 目录结构创建完成!


In [4]:
# 第二步：创建 core.py 内容
core_code = '''
"""
B站视频分析核心模块
"""
import asyncio
import time
from bilibili_api import video

class BilibiliAnalyzer:
    """B站视频分析器"""
    
    def __init__(self):
        self.sequence = []
        self.start_time = time.time()
    
    def _checkpoint(self, step_name):
        """内部断点检查"""
        timestamp = time.time() - self.start_time
        self.sequence.append(1)
        print(f"⏰ [{timestamp:.3f}s] {step_name}: 1")
        return timestamp
    
    async def analyze_video(self, bv_id):
        """分析视频数据"""
        try:
            # 创建视频对象
            self._checkpoint("创建视频对象")
            v = video.Video(bvid=bv_id)
            
            # 获取视频信息
            self._checkpoint("获取API数据")
            info = await v.get_info()
            
            # 提取数据
            self._checkpoint("解析数据")
            title = info['title']
            up_name = info['owner']['name']
            stat = info['stat']
            
            # 整理结果
            result = {
                'bv_id': bv_id,
                'title': title,
                'up_name': up_name,
                'view': stat['view'],
                'like': stat['like'],
                'coin': stat['coin'],
                'favorite': stat['favorite'],
                'share': stat['share'],
                'reply': stat['reply'],
                'sequence': self.sequence.copy(),
                'total_time': time.time() - self.start_time
            }
            
            # 计算比率
            result['like_rate'] = result['like'] / result['view']
            result['coin_rate'] = result['coin'] / result['view']
            result['favorite_rate'] = result['favorite'] / result['view']
            
            self._checkpoint("分析完成")
            return result
            
        except Exception as e:
            print(f"❌ 分析失败: {e}")
            return None

class InteractiveTester:
    """交互式测试器"""
    
    def __init__(self):
        self.analyzer = BilibiliAnalyzer()
    
    async def run_interactive_test(self):
        """运行交互式测试"""
        while True:
            print("\\n" + "="*50)
            bv_id = input("请输入BV号 (输入 'q' 退出): ").strip()
            
            if bv_id.lower() == 'q':
                print("测试结束！")
                break
                
            if not bv_id.startswith('BV'):
                print("❌ 请输入有效的BV号")
                continue
            
            print(f"🎯 正在分析: {bv_id}")
            result = await self.analyzer.analyze_video(bv_id)
            
            if result:
                self._display_result(result)
            else:
                print("❌ 分析失败")
    
    def _display_result(self, result):
        """显示分析结果"""
        print(f"\\n✅ 分析成功!")
        print(f"📺 标题: {result['title']}")
        print(f"👤 UP主: {result['up_name']}")
        print(f"📊 播放量: {result['view']:,}")
        print(f"👍 点赞: {result['like']:,} ({result['like_rate']:.2%})")
        print(f"🪙 投币: {result['coin']:,} ({result['coin_rate']:.2%})")
        print(f"⭐ 收藏: {result['favorite']:,} ({result['favorite_rate']:.2%})")
        print(f"🔢 执行序列: {''.join(str(x) for x in result['sequence'])}")
        print(f"⏱️  总耗时: {result['total_time']:.3f}s")
'''

with open('bilibili_analyzer/src/core.py', 'w', encoding='utf-8') as f:
    f.write(core_code)

print("✅ core.py 创建完成!")

✅ core.py 创建完成!


In [ ]:
# 第三步：测试封装好的包
from bilibili_analyzer.src.core import InteractiveTester

print("🚀 B站视频分析工具 - 封装版")
tester = InteractiveTester()
await tester.run_interactive_test()

🚀 B站视频分析工具 - 封装版



请输入BV号 (输入 'q' 退出):  BV1BNDNYkE7c


🎯 正在分析: BV1BNDNYkE7c
⏰ [15.880s] 创建视频对象: 1
⏰ [15.880s] 获取API数据: 1
⏰ [16.619s] 解析数据: 1
⏰ [16.619s] 分析完成: 1

✅ 分析成功!
📺 标题: 【天大航模队2024新生培训】第二课：linux基础操作与ROS入门
👤 UP主: TIM学生科技实验室
📊 播放量: 13,308
👍 点赞: 370 (2.78%)
🪙 投币: 48 (0.36%)
⭐ 收藏: 623 (4.68%)
🔢 执行序列: 111
⏱️  总耗时: 16.619s



请输入BV号 (输入 'q' 退出):  BV1BNDNYkE7c


🎯 正在分析: BV1BNDNYkE7c
⏰ [331.172s] 创建视频对象: 1
⏰ [331.172s] 获取API数据: 1
⏰ [331.245s] 解析数据: 1
⏰ [331.245s] 分析完成: 1

✅ 分析成功!
📺 标题: 【天大航模队2024新生培训】第二课：linux基础操作与ROS入门
👤 UP主: TIM学生科技实验室
📊 播放量: 13,308
👍 点赞: 370 (2.78%)
🪙 投币: 48 (0.36%)
⭐ 收藏: 623 (4.68%)
🔢 执行序列: 1111111
⏱️  总耗时: 331.245s



...

🎯 正在分析: BV1jYpTzWEpL
⏰ [2363.205s] 创建视频对象: 1
⏰ [2363.205s] 获取API数据: 1
⏰ [2363.280s] 解析数据: 1
⏰ [2363.280s] 分析完成: 1

✅ 分析成功!
📺 标题: CityPOP 🎵｜泡沫经济下的复古旋律 | 充满个人魅力与活力的古早日语歌 | 昭和日语旋律 𝑷𝒍𝒂𝒚𝒍𝒊𝒔𝒕
👤 UP主: LorryDriverRadio
📊 播放量: 17,386
👍 点赞: 875 (5.03%)
🪙 投币: 126 (0.72%)
⭐ 收藏: 1,300 (7.48%)
🔢 执行序列: 11111111111
⏱️  总耗时: 2363.280s



请输入BV号 (输入 'q' 退出):  BV13BnmzsEiY


🎯 正在分析: BV13BnmzsEiY
⏰ [2405.447s] 创建视频对象: 1
⏰ [2405.447s] 获取API数据: 1
⏰ [2405.525s] 解析数据: 1
⏰ [2405.525s] 分析完成: 1

✅ 分析成功!
📺 标题: 为什么说皇室战争取消宝箱倒计时是必然？
👤 UP主: CR_小记者皓皓
📊 播放量: 23,924
👍 点赞: 486 (2.03%)
🪙 投币: 4 (0.02%)
⭐ 收藏: 35 (0.15%)
🔢 执行序列: 111111111111111
⏱️  总耗时: 2405.525s



请输入BV号 (输入 'q' 退出):  BV1CVWwzREiZ


🎯 正在分析: BV1CVWwzREiZ
⏰ [2749.302s] 创建视频对象: 1
⏰ [2749.302s] 获取API数据: 1
⏰ [2749.363s] 解析数据: 1
⏰ [2749.363s] 分析完成: 1

✅ 分析成功!
📺 标题: 沉浸式体验十万名高中生抢饭! 丧尸级别！
👤 UP主: 我是七奇
📊 播放量: 1,873,347
👍 点赞: 168,018 (8.97%)
🪙 投币: 3,523 (0.19%)
⭐ 收藏: 26,111 (1.39%)
🔢 执行序列: 1111111111111111111
⏱️  总耗时: 2749.363s

